<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [2]:

# Подключим все необходимые библиотеки 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
df = pd.read_csv('/datasets/insurance.csv')

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Огонь, данные на месте:)</div>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
display(df.head())
display(df.tail())
display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
#приведем возраст и зарплату к int типу данных
df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)
df.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

In [7]:
#Посмотрим на количество дубликатов в таблице
df.duplicated().sum()


153

In [8]:
#Посмотрим на количество пустых значений
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [9]:
#Избавимся от дубликатов
df= df.drop_duplicates()

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Обрати внимание, что у нас нет однозначного идентификатора клиента типа id или ФИО, соответственно мы не можем со 100% гарантией утверждать, что все совпадения являются дубликатами</div>

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Ответ: b. Не изменится.



**Обоснование:** 

Если матрица P обратима, то при умножении матрицы признаков X на нее качество линейной регрессии не изменится. Это можно показать, обозначив новую матрицу признаков как $X' = XP$. Тогда новые веса $w'$ рассчитываются по формуле:

$$w' = (X'^TX')^{-1}X'^Ty = ((XP)^TXP)^{-1}(XP)^Ty = (P^TX^TXP)^{-1} P^T X^Ty = P^{-1}(XᵀX)^{-1}P^{T^{-1}}P^TX^Ty = P^{-1}w$$

То есть, новые веса $w'$ — это старые веса $w$, умноженные на обратную матрицу $P^{-1}$.

При подстановке $w'$ в формулу для предсказаний $a = X'w'$ получим:

$$a = X'w' = XPw' = XP(P^{-1}w) = Xw = a$$

то есть, предсказания $a$ не изменятся после преобразования признаков умножением на обратимую матрицу.

Таким образом, умножение признаков на обратимую матрицу не изменяет качество линейной регрессии.

Что и требовалось доказать.






<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Да, все абсолютно верно 10/10

P.S Чутка подправил степени, чтобы смотрелось получше
</div>

## Алгоритм преобразования

**Алгоритм**

Чтобы защитить данные и сохранить качество модели машинного обучения, нужно умножить признаки на обратимую матрицу. Для этого алгоритм преобразования данных должен содержать процесс создания подобной матрицы и ее умножения на матрицу признаков.



In [10]:
#Так как по условию задачи эта матрица при умножении на матрицу признаков должна давать матрицу такого же размера
#как исходная, то $P$ должна быть квадратной с размером, равным количеству признаков, в нашем случае их 4.
random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[12, 21, 35, 40],
       [80, 95, 34, 20],
       [76, 73, 81, 32],
       [43, 92, 97, 65]])

Обратимая (или невырожденная) матрица - это такая квадратная матрица, для которой существует обратная матрица. Если матрицу A умножить на её обратную матрицу (или обратную матрицу умножить на матрицу A), то результатом будет единичная матрица. Единичная матрица - это матрица, у которой на диагонали стоят единицы, а все остальные элементы равны нулю.

Важно отметить, что обратная матрица существует не для всех квадратных матриц. Например, матрица, у которой определитель (детерминант) равен нулю, называется вырожденной, и у неё нет обратной матрицы.

В Python, чтобы проверить, обратима ли матрица, можно вычислить её определитель с помощью функции numpy.linalg.det(). Если определитель не равен нулю, то матрица обратима.

In [20]:
# Вычисляем определитель
det = np.linalg.det(random_matrix)
if det != 0:
    print("Матрица обратима")
else:
    print("Матрица не обратима")

Матрица обратима


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Верно проверяешь на обратимость</div>

In [18]:
# Находим обратную матрицу
A_inv = np.linalg.inv(random_matrix)

In [19]:
A_inv

array([[ 0.02468472,  0.00587017,  0.0176002 , -0.02566152],
       [-0.02339178,  0.01025247, -0.01865432,  0.020424  ],
       [-0.02121599, -0.01822581,  0.01659848,  0.01049238],
       [ 0.04843926,  0.00880398, -0.01001021, -0.01220497]])

нам известно, что предсказания модели при преобразовании не поменяются, а значит качество модели тоже не должно меняться, так как качество модели - это производная от предсказаний (которые одинаковы до и после преобразований) и целевого признака.

Подкрепим теоритические рассуждения примером на цифрах из таблицы. Для этого самостоятельно расчитаем вектора предсказаний 
 и 
 по формулам и сравним их между собой.

Сначала выделим признаки и целевой признак:


In [12]:
#Разделим датафрейм df на признаки (features) и целевой признак (target). 
#Целевым признаком в нашем случае являются страховые выплаты
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']
 
#С помощью StandardScaler() происходит масштабирование признаков. Это процесс, при котором все числовые признаки приводятся 
#к единому масштабу, чтобы ни один из признаков не доминировал над остальными при построении модели. 
#Обычно это делается путем вычета среднего значения и деления на стандартное отклонение.
scaler = StandardScaler()
scaler.fit(features)

#Сохраним масштабированные признаки в features_scaled
features_scaled = scaler.transform(features)

Процесс проверки гипотезы о том, что преобразование признаков не сильно влияет на предсказания модели.

In [21]:
#С помощью матричных операций вычислим веса w для каждого признака в датасете. Это делается на основе формулы линейной регрессии.
#Расчитаем веса $w$ для исходных признаков (масштабированных) по стандартной формуле $w = (X^T X)^{-1} X^T y$
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

#Расчитаем вектор предсказаний по исходным признакам по формуле 𝑎=𝑋𝑤
#Используем вектор весов w для предсказания целевого признака a для исходных (масштабированных) признаков. Умножим матрицу на вектор:
a = features_scaled @ w

#Проведем преобразование признаков путем умножения на случайную матрицу random_matrix
features_scaled_p = features_scaled @ random_matrix

#Расчитаем веса  𝑤1  для преобразованных признаков по стандартной формуле  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦
w1 = np.linalg.inv(features_scaled_p.T.dot(features_scaled_p)).dot(features_scaled_p.T).dot(target)

#Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤
a1 = features_scaled_p @ w1

#Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1 :
diff = a - a1

#Просуммируем все элементы вектора diff, чтобы убедиться, что расхождения минимальны.
diff.sum()

9.22543833860312e-14

Видно, что расхождения небольшие. Это значит 𝑎 почти равен 𝑎1. Преобразование не приводит к разным предсказаниям модели и не ухудшает качество модели.

## Проверка алгоритма

$$
w = (X^T X)^{-1} X^T y
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$

In [22]:
#Создадим модель линейной регрессии для исходных данных и посмотрим ее оценку R2_Score
model = LinearRegression(normalize = True)
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

MSE Score: 0.1252726382276536
R2_Score на исходных данных:  0.4302010046633359


In [23]:
#Преобразуем признаки умножив на матрицу P и рассчитаем R2_Score
features_matrix = features_scaled @ random_matrix
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_matrix))

MSE Score: 0.1252726382276536
R2_Score для преобразованных  признаков:  0.4302010046633359


Видно, что качество модели, после умножения исходных признаков на случайную обратимую матрицу, не изменилось и подобное умножение не повлияло на качество предсказания модели, а значит мы защитили таким образом исходные даннные.

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Да, хорошо;)


Оставлю немного полезного материала:

+  https://academy.yandex.ru/handbook/ml
+  https://habr.com/ru/post/595281/
</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>
<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>Андрей, спасибо за хороший проект!!! Я готов принять работу, но хочу убедиться, что тебе все понятно.<br>
Если есть какие либо вопросы я с удовольствием на них отвечу:)</div>


<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b>

Оставлю хороший учебник по ML:
+  https://academy.yandex.ru/handbook/ml
    
Немного новостей с мира DL(вдруг заинтересует):
+  https://habr.com/ru/company/ods/blog/686962/
    
Вот тут проходят всякие соревнования, можешь себя попробовать:
+  https://hacks-ai.ru/championships#competitions
    

Возможно, что нить пригодится:  
+  https://drive.google.com/file/d/1r8CJHH_hwDM16l1Pnpb4KJzoJFrJ5XRi/view
+  https://arxiv.org/ftp/arxiv/papers/2201/2201.00650.pdf
+  https://github.com/slgero/testovoe
    
    
Курс от МФТИ математика для DS:
    
+  https://www.youtube.com/watch?v=xccjt6lOoow&list=PLk4h7dmY2eYHHTyfLyrl7HmP-H3mMAW08&index=1
    
    
Тут много соревнований/лекций:
    
+  https://ods.ai/
    
stepic NLP/CV:
    
+  https://stepik.org/course/54098/syllabus
+  https://stepik.org/course/50352/syllabus
    
Еще есть от ВШЭ:
    
+  https://www.youtube.com/watch?v=mwjQaNt8qxk&list=PLEwK9wdS5g0og-DcF1apxutSM0GDLHz_3&ab_channel=%D0%A4%D0%9A%D0%9D%D0%92%D0%A8%D0%AD%E2%80%94%D0%B4%D0%B8%D1%81%D1%82%D0%B0%D0%BD%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8F
+  https://github.com/hse-ds/iad-deep-learning
    
    
    
    
Удачи в следующих проектах!!!
    
 


</div>